# Searching in a transformed space

Helps, not sure why

Link to pre-print

Link to figure from four-way fitting


Show some complex figure, then add a few log-transforms to it, watch it calm down?


## Implementing

In [1]:
# Example code, based on four-ways code

## Summary